## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import config from another folder
import sys
sys.path.append('../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Maceio,BR,-9.6658,-35.7353,82.40,65,75,11.50,broken clouds,2021-03-25 19:13:48
1,1,Clyde River,CA,70.4692,-68.5914,-4.00,84,75,12.66,snow,2021-03-25 19:14:20
2,2,Klaksvik,FO,62.2266,-6.5890,44.60,87,90,12.66,overcast clouds,2021-03-25 19:06:08
3,3,Chuy,UY,-33.6971,-53.4616,74.14,81,100,11.48,moderate rain,2021-03-25 19:14:20
4,4,Busselton,AU,-33.6500,115.3333,51.01,93,25,8.08,scattered clouds,2021-03-25 19:14:20


In [18]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [19]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_cities = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

In [20]:
# Determine if there are any empty rows.
user_cities.count()

City_ID                246
City                   246
Country                240
Lat                    246
Lng                    246
Max Temp               246
Humidity               246
Cloudiness             246
Wind Speed             246
Current Description    246
Date                   246
dtype: int64

In [21]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = user_cities.dropna()

In [22]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.NaN
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Chuy,UY,74.14,moderate rain,-33.6971,-53.4616,NaN
6,Albany,US,70.00,scattered clouds,42.6001,-73.9662,NaN
7,Port Alfred,ZA,70.47,broken clouds,-33.5906,26.8910,NaN
11,Dingle,PH,74.82,overcast clouds,10.9995,122.6711,NaN
12,Hilo,US,77.00,broken clouds,19.7297,-155.0900,NaN
13,Hermanus,ZA,71.01,broken clouds,-34.4187,19.2345,NaN
14,Mar Del Plata,AR,71.01,light rain,-38.0023,-57.5575,NaN
15,Kapaa,US,73.40,few clouds,22.0752,-159.3190,NaN
16,Nakhon Phanom,TH,75.45,scattered clouds,17.3333,104.4167,NaN
17,Saldanha,ZA,69.80,broken clouds,-33.0117,17.9442,NaN


In [23]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Chuy,UY,74.14,moderate rain,-33.6971,-53.4616,Nuevo Hotel Plaza
7,Port Alfred,ZA,70.47,broken clouds,-33.5906,26.8910,The Halyards Hotel
11,Dingle,PH,74.82,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
12,Hilo,US,77.00,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Hermanus,ZA,71.01,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
14,Mar Del Plata,AR,71.01,light rain,-38.0023,-57.5575,Gran Hotel Mar del Plata
15,Kapaa,US,73.40,few clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
16,Nakhon Phanom,TH,75.45,scattered clouds,17.3333,104.4167,ป่ายางรีสอร์ท
17,Saldanha,ZA,69.80,broken clouds,-33.0117,17.9442,Blue Bay Lodge
18,Vao,NC,77.09,scattered clouds,-22.6667,167.4833,Hotel Kou-Bugny


In [25]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
        <dl>
        <dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
        <dt><b>City</b></dt><dd>{City}</dd>
        <dt><b>Country</b></dt><dd>{Country}</dd>
        <dt><b>Weather description</b></dt><dd>{Current Description} and max {Max Temp} °F</dd>
        </dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
list_locations = list(locations.to_records(index=False))

In [37]:
# Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(list_locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))